In [1]:
# ============================================================
# 🔧 INSTALLATION DES DEPENDANCES
# ============================================================
!pip install transformers datasets sentencepiece accelerate evaluate sacrebleu -q
!pip install torch torchvision torchaudio -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.

In [2]:
!pip install sacrebleu

In [3]:
!pip install evaluate

In [4]:
# ============================================================
# 📚 IMPORTS ET CONFIGURATION DE BASE
# ============================================================
# ============================================================
# 1) Imports & device
# ============================================================
import os
import math
import json
from pathlib import Path
from typing import List, Dict, Any


import numpy as np
import pandas as pd


import torch
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (
AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments,
DataCollatorForTokenClassification, DataCollatorForSeq2Seq,
AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
)
import evaluate


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)

2025-11-15 20:30:45.925279: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763238646.105594      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763238646.156605      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Device: cuda


In [5]:
# ============================================================
# 📂 CHARGEMENT DU DATASET
# ============================================================

# Si tu as ton dataset local
# uploaded = files.upload()
df = pd.read_csv("/kaggle/input/dataset-autocorrection/automatic_correction_dataset.csv")

df.head()

,Wrong,Right
0,I WANT TO THAK YOU FOR PREPARING SUCH A GOOD...,I WANT TO THANK YOU FOR PREPARING SUCH A GOO...
1,IT CONSISTS ON INVOLVES VISITING THE LONDON F...,IT IN INVOLVES VISITING THE LONDON FASHION AN...
2,"ON THE OTHER HAND , WE COULD LEARN THE DIFFERE...","ALSO , WE COULD LEARN THE DIFFERENT WAYS TO GE..."
3,I WILL BE WRITING ANXIOUSLY FOR YOUR RESPONSE.,I WILL BE WAITING ANXIOUSLY FOR YOUR RESPONSE.
4,"AS SOME OF THEM ARE CONSIDERED IDOLS, THEIR FA...","AS SOME OF THEM ARE CONSIDERED IDOLS, THEIR FA..."


In [6]:
# ============================================================
# 🔀 SPLIT DU DATASET
# ============================================================
from sklearn.model_selection import train_test_split

# 80% train, 10% val, 10% test
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_ds = Dataset.from_pandas(train_df)
val_ds   = Dataset.from_pandas(val_df)
test_ds  = Dataset.from_pandas(test_df)

dataset = DatasetDict({
    "train": train_ds,
    "validation": val_ds,
    "test": test_ds
})

print(dataset)


DatasetDict({
    train: Dataset({
        features: ['Wrong', 'Right', '__index_level_0__'],
        num_rows: 14952
    })
    validation: Dataset({
        features: ['Wrong', 'Right', '__index_level_0__'],
        num_rows: 1869
    })
    test: Dataset({
        features: ['Wrong', 'Right', '__index_level_0__'],
        num_rows: 1870
    })
})


In [7]:
train_ds

Dataset({
    features: ['Wrong', 'Right', '__index_level_0__'],
    num_rows: 14952
})

In [8]:
# ============================================================
# 🧠 CHOIX DU MODELE ET TOKENIZER
# ============================================================

model_name = "t5-base"  # ou "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_name)



# Préparation du modèle
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
# ============================================================
# ✂️ TOKENISATION
# ============================================================
def preprocess_function(examples):
    inputs = examples["Wrong"]                 # phrase incorrecte
    targets = examples["Right"]                # phrase corrigée

    model_inputs = tokenizer(
        inputs, max_length=128, truncation=True
    )

    labels = tokenizer(
        text_target=targets, max_length=128, truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



In [10]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/14952 [00:00<?, ? examples/s]

Map:   0%|          | 0/1869 [00:00<?, ? examples/s]

Map:   0%|          | 0/1870 [00:00<?, ? examples/s]

In [11]:
print(tokenized_datasets["train"][0]["labels"])

[94, 66, 2817, 3, 9, 215, 977, 6, 116, 5192, 11, 27, 130, 341, 200, 803, 11, 261, 12, 817, 284, 119, 762, 5, 1]


In [12]:
print(tokenized_datasets["train"].features)

{'Wrong': Value('string'), 'Right': Value('string'), '__index_level_0__': Value('int64'), 'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8')), 'labels': List(Value('int64'))}


In [13]:
tokenized_datasets["train"][0]


{'Wrong': 'I  all happened a year ago, when Pat and I were still best friends and used to tell each other everything.',
 'Right': 'It  all happened a year ago, when Pat and I were still best friends and used to tell each other everything.',
 '__index_level_0__': 16394,
 'input_ids': [27,
  66,
  2817,
  3,
  9,
  215,
  977,
  6,
  116,
  5192,
  11,
  27,
  130,
  341,
  200,
  803,
  11,
  261,
  12,
  817,
  284,
  119,
  762,
  5,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [94,
  66,
  2817,
  3,
  9,
  215,
  977,
  6,
  116,
  5192,
  11,
  27,
  130,
  341,
  200,
  803,
  11,
  261,
  12,
  817,
  284,
  119,
  762,
  5,
  1]}

In [14]:
# ============================================================
# 🎓 ENTRAINEMENT DU MODELE
# ============================================================

import os
os.environ["WANDB_DISABLED"] = "true" # This line is deprecated, setting report_to="none" in training_args is preferred

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
metric = evaluate.load("sacrebleu")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) # Fix: Replace -100 in labels before decoding
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return {"bleu": metric.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])["score"]}

training_args = Seq2SeqTrainingArguments(
    output_dir="./corrector-model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=50,
    push_to_hub=False,
    report_to="none" # Fix: Explicitly disable reporting to external services like Weights & Biases
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipykernel_48/4120385583.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Bleu
1,0.366000,0.345350,60.149828
2,0.348400,0.325686,60.883650
3,0.319600,0.323248,61.013879


TrainOutput(global_step=11214, training_loss=0.37497413088256115, metrics={'train_runtime': 2565.9808, 'train_samples_per_second': 17.481, 'train_steps_per_second': 4.37, 'total_flos': 1952309482168320.0, 'train_loss': 0.37497413088256115, 'epoch': 3.0})

In [16]:
def clean_corrected_text(text):
    """
    Nettoie la sortie du modèle pour ne garder que la phrase corrigée.
    """

    # Liste de préfixes possibles que le modèle peut renvoyer
    prefixes_to_remove = [
        "Corrige la phrase suivante :",
        "corrige la phrase suivante :",
        "Corrige la phrase suivante:",
        "corrige la phrase suivante:",
        "Correction :",
        "correction :",
        "Voici la phrase corrigée :",
        "Phrase corrigée :",
    ]

    cleaned = text.strip()

    # On enlève tous les préfixes potentiels
    for prefix in prefixes_to_remove:
        if cleaned.lower().startswith(prefix.lower()):
            cleaned = cleaned[len(prefix):].strip()

    # Nettoyage espaces multiples
    cleaned = " ".join(cleaned.split())

    return cleaned


In [17]:
# ============================================================
# 🧪 TEST SUR JEU DE TEST
# ============================================================

def correct_sentence(sentence):
    input_text = f"Corrige la phrase suivante : {sentence}"
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=128)
    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return clean_corrected_text(raw_output)


# On récupère les phrases du dataset test
test_wrong = test_df["Wrong"].tolist()
test_right = test_df["Right"].tolist()

results = []

for wrong, right in zip(test_wrong, test_right):
    corrected = correct_sentence(wrong)
    results.append((wrong, corrected, right))

# Affichage propre
for wrong, corrected, right in results[:20]:  # Affiche les 20 premiers
    print("❌ Phrase incorrecte :", wrong)
    print("✅ Correction modèle :", corrected)
    print("🎯 Correction attendue :", right)
    print("-" * 80)


❌ Phrase incorrecte : First of all, you must walk down  to the Central Street until you arrive at the bus stop, then you take the bus number two  that goes directly to the college.
✅ Correction modèle : First of all, you must walk down to the Central Street until you arrive at the bus stop, then you take the bus number two that goes directly to the college.
🎯 Correction attendue : First of all, you must walk down  number two bus  that goes directly to the college.
--------------------------------------------------------------------------------
❌ Phrase incorrecte : Brown ,  I hope he will get better soon.
✅ Correction modèle : Brown , I hope he will get better soon.
🎯 Correction attendue : Brown .  I hope he will get better soon.
--------------------------------------------------------------------------------
❌ Phrase incorrecte : The  Rules  at school are: we must be on time  at  lessons, and we  always have must always  make  our homework.
✅ Correction modèle : The rules at school ar

In [18]:
# ============================================================
# 💾 SAUVEGARDE DU MODELE ET TOKENIZER
# ============================================================
model.save_pretrained("./final_corrector_model")
tokenizer.save_pretrained("./final_corrector_model")
print("✅ Modèle et tokenizer sauvegardés !")


✅ Modèle et tokenizer sauvegardés !


In [19]:
import pickle

save_path = "./final_corrector_model"

pickle_data = {
    "model_name": "t5-base",
    "save_path": save_path
}

with open("model_info.pkl", "wb") as f:
    pickle.dump(pickle_data, f)

print("📦 Pickle saved successfully!")


📦 Pickle saved successfully!


In [18]:
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM

# Charger modèle HF en version TensorFlow
tf_model = TFAutoModelForSeq2SeqLM.from_pretrained(save_path, from_pt=True)

# Construire un modèle Keras wrapper
input_ids = tf.keras.Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(128,), dtype=tf.int32, name="attention_mask")

outputs = tf_model({"input_ids": input_ids, "attention_mask": attention_mask}).logits

keras_model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)


I0000 00:00:1763199797.549427      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10313 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


ValueError: Exception encountered when calling layer "tft5_for_conditional_generation" (type TFT5ForConditionalGeneration).

in user code:

    File "/usr/local/lib/python3.11/dist-packages/transformers/modeling_tf_utils.py", line 1395, in run_call_with_unpacked_inputs  *
        return func(self, **unpacked_inputs)
    File "/usr/local/lib/python3.11/dist-packages/transformers/models/t5/modeling_tf_t5.py", line 1455, in call  *
        decoder_outputs = self.decoder(
    File "/usr/local/lib/python3.11/dist-packages/tf_keras/src/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_file_vbmqzf6.py", line 40, in tf__run_call_with_unpacked_inputs
        raise
    File "/tmp/__autograph_generated_file3nyaiess.py", line 65, in tf__call
        ag__.if_stmt(ag__.and_(lambda: ag__.ld(input_ids) is not None, lambda: ag__.ld(inputs_embeds) is not None), if_body_2, else_body_2, get_state_2, set_state